In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap

from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM, VQC
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

seed = 10599
aqua_globals.random_seed = seed

import sys
import os
from pathlib import Path
import pickle
import numpy as np
import time
import sklearn.model_selection as model_selection
from sklearn.decomposition import PCA
main_folder=str(Path.cwd().parent) 
sys.path.append(main_folder) 
data_folder = f'{main_folder}/data'

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

<ipython-input-1-1d6f60b255b6>:12: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.algorithm_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  aqua_globals.random_seed = seed


In [2]:
# from qiskit import IBMQ

# # IBMQ.save_account(TOKEN)
# IBMQ.load_account() # Load account from disk
# IBMQ.providers()    # List all available providers


In [3]:
# # choose backend
# provider = IBMQ.get_provider("ibm-q")

# for backend in provider.backends():
#   try:
#     qubit_count = len(backend.properties().qubits)
#   except:
#     qubit_count = "simulated"

#   print(f"{backend.name()} has {backend.status().pending_jobs} queued and {qubit_count} qubits")

# TO-DO:

- Improve the accuracy via Feature Engineer or Hyperparam
- Try with 3 classes (STAR, GALAXY , QSOR)
- Try with more data (currently training is 100 and test is 40 and test extra is 20)

## Loading the processed data DR16_Processed

In [4]:
def split_balanced(data, target, train_size, test_size):
    
    np.random.seed(0)

    classes = np.unique(target)
    # can give test_size as fraction of input data size of number of samples
    if test_size<1:
        n_test = np.round(len(target)*test_size)
    else:
        n_test = test_size
    n_train = train_size #max(0,len(target)-n_test)
    n_train_per_class = max(1,int(np.floor(n_train/len(classes))))
    n_test_per_class = max(1,int(np.floor(n_test/len(classes))))

    ixs = []
    for cl in classes:
        if (n_train_per_class+n_test_per_class) > np.sum(target==cl):
            # if data has too few samples for this class, do upsampling
            # split the data to training and testing before sampling so data points won't be
            #  shared among training and test data
            splitix = int(np.ceil(n_train_per_class/(n_train_per_class+n_test_per_class)*np.sum(target==cl)))
            ixs.append(np.r_[np.random.choice(np.nonzero(target==cl)[0][:splitix], n_train_per_class),
                np.random.choice(np.nonzero(target==cl)[0][splitix:], n_test_per_class)])
        else:
            ixs.append(np.random.choice(np.nonzero(target==cl)[0], n_train_per_class+n_test_per_class,
                replace=False))

    # take same num of samples from all classes
    ix_train = np.concatenate([x[:n_train_per_class] for x in ixs])
    ix_test = np.concatenate([x[n_train_per_class:(n_train_per_class+n_test_per_class)] for x in ixs])

    X_train = data[ix_train,:]
    X_test = data[ix_test,:]
    y_train = target[ix_train]
    y_test = target[ix_test]

    return X_train, X_test, y_train, y_test

In [5]:
prep = ZZFeatureMap(2, reps=2)
print(prep)

     ┌───┐┌──────────────┐                                           ┌───┐»
q_0: ┤ H ├┤ U1(2.0*x[0]) ├──■─────────────────────────────────────■──┤ H ├»
     ├───┤├──────────────┤┌─┴─┐┌───────────────────────────────┐┌─┴─┐├───┤»
q_1: ┤ H ├┤ U1(2.0*x[1]) ├┤ X ├┤ U1(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ H ├»
     └───┘└──────────────┘└───┘└───────────────────────────────┘└───┘└───┘»
«     ┌──────────────┐                                           
«q_0: ┤ U1(2.0*x[0]) ├──■─────────────────────────────────────■──
«     ├──────────────┤┌─┴─┐┌───────────────────────────────┐┌─┴─┐
«q_1: ┤ U1(2.0*x[1]) ├┤ X ├┤ U1(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
«     └──────────────┘└───┘└───────────────────────────────┘└───┘


In [6]:
# Load the transofrmed data 
with open(f'{data_folder}/processed/DR16_processed_X.pkl','rb') as input_file:
    X = pickle.load(input_file)
with open(f'{data_folder}/processed/DR16_processed_y.pkl','rb') as input_file:
    y = pickle.load(input_file)

In [7]:
y.values

array(['STAR', 'STAR', 'STAR', ..., 'GALAXY', 'GALAXY', 'STAR'],
      dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_num = le.fit_transform(y.values)

In [9]:
set(y)

{'GALAXY', 'QSO', 'STAR'}

### Imbalanced Dataset

In [10]:
#Create training_dataset and test_dataset with STAR and NOT_STAR
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_num, train_size=0.001, test_size=0.0006, random_state=101, stratify=y_num)

X_val = X_test[0:40]
X_pred = X_test[40:60]
y_val = y_test[0:40]
y_pred = y_test[40:60]

print(X_train.shape)
print(X_val.shape)
print(X_pred.shape)


print(y_train.shape)
print(y_val.shape)
print(y_pred.shape)


(100, 8)
(40, 8)
(20, 8)
(100,)
(40,)
(20,)


In [11]:
(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[ 0, 51],
       [ 1, 11],
       [ 2, 38]])

### Balanced Dataset

In [12]:
train_samples = 200 
test_samples  = 150

In [13]:
X_train, X_test, y_train, y_test = split_balanced(np.array(X), y_num, train_size=train_samples, test_size=test_samples)

print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)



(198, 8)
(150, 8)
(198,)
(150,)


In [14]:
(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(" Training data distribution\n", frequencies)

(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(" Test data distribution\n", frequencies)

 Training data distribution
 [[ 0 66]
 [ 1 66]
 [ 2 66]]
 Test data distribution
 [[ 0 50]
 [ 1 50]
 [ 2 50]]


In [15]:
class_labels = list(set(y))
class_labels

['GALAXY', 'STAR', 'QSO']

## Dimensionality Reduction

## No Reduction

In [16]:
# # Create datapoint[X_test,y]
datapoints = [np.array(X_test),np.array(y_test)]
# If class labels are numeric
training_size = len(X_train)
test_size = len(X_test)

#this is where I transform our dataframe to Dict[key:np.array]
if class_labels[0].isdigit():
        # Pick training size number of samples from each distro
    training_input = {key: (X_train[y_train == int(key), :])[:training_size] for k, key in enumerate(class_labels)}
    test_input = {key: (X_test[y_test == int(key), :])[: test_size] for k, key in enumerate(class_labels)}
else:
    # if they aren't
    training_input = {key: (X_train[y_train == k, :])[:training_size] for k, key in enumerate(class_labels)}
    test_input = {key: (X_test[y_test == k, :])[:test_size] for k, key in enumerate(class_labels)}

## PCA

In [17]:
# pca = PCA(n_components=3).fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

# # Create datapoint[X_test,y]
# datapoints = [np.array(X_test),np.array(y_test)]
# # If class labels are numeric
# training_size = len(X_train)
# test_size = len(X_test)

# #this is where I transform our dataframe to Dict[key:np.array]
# if class_labels[0].isdigit():
#         # Pick training size number of samples from each distro
#     training_input = {key: (X_train[y_train == int(key), :])[:training_size] for k, key in enumerate(class_labels)}
#     test_input = {key: (X_test[y_test == int(key), :])[: test_size] for k, key in enumerate(class_labels)}
# else:
#     # if they aren't
#     training_input = {key: (X_train[y_train == k, :])[:training_size] for k, key in enumerate(class_labels)}
#     test_input = {key: (X_test[y_test == k, :])[:test_size] for k, key in enumerate(class_labels)}


In [18]:
# import plotly.express as px

# total_var = pca.explained_variance_ratio_.sum() * 100
# print(total_var)

# labels = {
#     str(i): f"PC {i+1} ({var:.1f}%)"
#     for i, var in enumerate(pca.explained_variance_ratio_ * 100)
# }

# fig = px.scatter(X_train, x=0, y=1, color=y_train)
# fig.show()

In [19]:
# total_var = pca.explained_variance_ratio_.sum() * 100

# fig = px.scatter_3d(
#     X_train, x=0, y=1, z=2, color=y_train,
#     title=f'Total Explained Variance: {total_var:.2f}%',
#     labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
# )
# fig.show()

## ISOMap

In [20]:
from sklearn.manifold import Isomap, LocallyLinearEmbedding
iso_embedding = Isomap(n_components=5, n_jobs = 4, n_neighbors = 5)
iso_embedding.fit(X_train[0:5000,:])
X_train_iso = iso_embedding.transform(X_train)
X_test_iso = iso_embedding.transform(X_test)

# Create datapoint[X_test_iso,y]
datapoints = [np.array(X_test_iso),np.array(y_test)]
# If class labels are numeric
training_size = len(X_train_iso)
test_size = len(X_test_iso)

#this is where I transform our dataframe to Dict[key:np.array]
if class_labels[0].isdigit():
        # Pick training size number of samples from each distro
    training_input_iso = {key: (X_train_iso[y_train == int(key), :])[:training_size] for k, key in enumerate(class_labels)}
    test_input_iso = {key: (X_test_iso[y_test == int(key), :])[: test_size] for k, key in enumerate(class_labels)}
else:
    # if they aren't
    training_input_iso = {key: (X_train_iso[y_train == k, :])[:training_size] for k, key in enumerate(class_labels)}
    test_input_iso = {key: (X_test_iso[y_test == k, :])[:test_size] for k, key in enumerate(class_labels)}


## Modified Locally Linear Embedding



In [21]:
# lle_embedding = LocallyLinearEmbedding(n_components=3, n_neighbors = 10, n_jobs = 4,  random_state=2019)
# lle_embedding.fit(X_train[:5000,:])
# X_train_lle = lle_embedding.transform(X_train)
# X_test_lle = lle_embedding.transform(X_test)

# # Create datapoint[X_test_lle,y]
# datapoints = [np.array(X_test_lle),np.array(y_test)]
# # If class labels are numeric
# training_size = len(X_train_lle)
# test_size = len(X_test_lle)

# #this is where I transform our dataframe to Dict[key:np.array]
# if class_labels[0].isdigit():
#         # Pick training size number of samples from each distro
#     training_input_lle = {key: (X_train_lle[y_train == int(key), :])[:training_size] for k, key in enumerate(class_labels)}
#     test_input_lle = {key: (X_test_lle[y_test == int(key), :])[: test_size] for k, key in enumerate(class_labels)}
# else:
#     # if they aren't
#     training_input_lle = {key: (X_train_lle[y_train == k, :])[:training_size] for k, key in enumerate(class_labels)}
#     test_input_lle = {key: (X_test_lle[y_test == k, :])[:test_size] for k, key in enumerate(class_labels)}

### QSVM

Initialisation

In [22]:
from qiskit.aqua.components.multiclass_extensions import AllPairs, OneAgainstRest, ErrorCorrectingCode

seed = 10598
feature_dim = 5

Hypertune QSVM algos

In [23]:
# #ISO
# feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
# qsvm = QSVM(feature_map, training_input_iso, test_input_iso, multiclass_extension = ErrorCorrectingCode())

# backend = BasicAer.get_backend('qasm_simulator')
# quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)
# start_run = time.time()
# result_ECC_zxzy_iso = qsvm.run(quantum_instance)
# end_run = time.time()  
# total_run_time = end_run - start_run
# print("testing success ratio: ", result_ECC_zxzy_iso['testing_accuracy'])
# print(f'run time for QSVM_iso is {total_run_time}')

In [24]:
# # Trial 1
# feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
# qsvm = QSVM(feature_map, training_input, test_input, multiclass_extension = AllPairs())

# backend = BasicAer.get_backend('qasm_simulator')
# quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)
# start_run = time.time()
# result_AP_zxzy_noDR = qsvm.run(quantum_instance)
# end_run = time.time()  
# total_run_time = end_run - start_run
# print("testing success ratio: ", result_AP_zxzy_noDR['testing_accuracy'])
# print(f'run time for QSVM AP without Dimension Reduction is {total_run_time}')

In [25]:
# Trial 2
feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
qsvm = QSVM(feature_map, training_input, test_input, multiclass_extension = ErrorCorrectingCode())

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start_run = time.time()
result_ECC_zxzy_noDR = qsvm.run(quantum_instance)
end_run = time.time()  
total_run_time = end_run - start_run
print("testing success ratio: ", result_ECC_zxzy_noDR['testing_accuracy'])
print(f'run time for QSVM ECC zwithout Dimension Reduction is {total_run_time}')

/Users/roro/Documents/QLearning/Quantstellar-QiskitHackathon21/QUANTSTELLAR-QISKIT-HACKATHON-21/venv/lib/python3.8/site-packages/qiskit/aqua/components/multiclass_extensions/multiclass_extension.py:34: DeprecationWarning: The package qiskit.aqua.components.multiclass_extensions is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.multiclass_extensions')
/Users/roro/Documents/QLearning/Quantstellar-QiskitHackathon21/QUANTSTELLAR-QISKIT-HACKATHON-21/venv/lib/python3.8/site-packages/qiskit/aqua/algorithms/classifiers/qsvm/qsvm.py:102: DeprecationWarning: The package qiskit.aqua.algorithms.classifiers is deprecated. It was moved/refactored to qiskit_machine_learning.algorithms.classifiers (pip install qiskit-machine-learning). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.algorithms.classifiers',
/Users/roro/Documents

In [26]:
# Trial 3
feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
qsvm = QSVM(feature_map, training_input, test_input, multiclass_extension = OneAgainstRest())

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start_run = time.time()
result_OAR_zxzy_noDR = qsvm.run(quantum_instance)
end_run = time.time()  
total_run_time = end_run - start_run
print("testing success ratio: ", result_OAR_zxzy_noDR['testing_accuracy'])
print(f'run time for QSVM OAQ without Dimension Reduction is {total_run_time}')

testing success ratio:  0.8
run time for QSVM OAQ without Dimension Reduction is 10494.88777589798


In [ ]:
# Trial 4
feature_map =  PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z', 'ZZ'])
qsvm = QSVM(feature_map, training_input, test_input, multiclass_extension = ErrorCorrectingCode())

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

start_run = time.time()
result_ECC_zzz_noDR = qsvm.run(quantum_instance)
end_run = time.time()  
total_run_time = end_run - start_run
print("testing success ratio: ", result_ECC_zzz_noDR['testing_accuracy'])
print(f'run time for QSVM ECC ZZZ without Dimension Reduction is {total_run_time}')

## Running jobs on IBM Q Server

In [23]:
from qiskit import IBMQ

IBMQ.load_account()

IBMQ.stored_account()

IBMQ.active_account()

provider=IBMQ.get_provider('ibm-q')

simulator_backend = provider.get_backend('ibmq_qasm_simulator')

In [24]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') fr

In [25]:
feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
qsvm = QSVM(feature_map, training_input_iso, test_input_iso, multiclass_extension = ErrorCorrectingCode())

#backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(simulator_backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)
start_run = time.time()
result_AP_zxzy_noDR = qsvm.run(quantum_instance)
end_run = time.time()  
total_run_time = end_run - start_run
print("testing success ratio: ", result_AP_zxzy_noDR['testing_accuracy'])
print(f'run time for QSVM AP without Dimension Reduction is {total_run_time}')

/Users/roro/Documents/QLearning/Quantstellar-QiskitHackathon21/QUANTSTELLAR-QISKIT-HACKATHON-21/venv/lib/python3.8/site-packages/qiskit/aqua/components/multiclass_extensions/multiclass_extension.py:34: DeprecationWarning: The package qiskit.aqua.components.multiclass_extensions is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.multiclass_extensions')
/Users/roro/Documents/QLearning/Quantstellar-QiskitHackathon21/QUANTSTELLAR-QISKIT-HACKATHON-21/venv/lib/python3.8/site-packages/qiskit/aqua/algorithms/classifiers/qsvm/qsvm.py:102: DeprecationWarning: The package qiskit.aqua.algorithms.classifiers is deprecated. It was moved/refactored to qiskit_machine_learning.algorithms.classifiers (pip install qiskit-machine-learning). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.algorithms.classifiers',
/Users/roro/Documents

KeyboardInterrupt: 

## Classical SVM

Three-class classification

In [ ]:
from qiskit.aqua.algorithms import SklearnSVM

start_run = time.time()
result_svm_ECC_noDR = SklearnSVM(training_input, test_input, multiclass_extension =  ErrorCorrectingCode()).run()
end_run = time.time()  
total_run_time = end_run - start_run
print(f'Testing success ratio of SVM without Dimension Reduction {result_svm_ECC_noDR["testing_accuracy"]}')
print(f'run time for SVM ECC without Dimension Reduction is {total_run_time}')

result_svm_AP_noDR = SklearnSVM(training_input , test_input , multiclass_extension =  AllPairs()).run()
end_run = time.time()  
total_run_time = end_run - start_run
print(f'Testing success ratio of SVM AP: {result_svm_AP_noDR ["testing_accuracy"]}')
print(f'run time for SVM AP is {total_run_time}')

In [1]:
# Saving Result 

# with open(f'{main_folder}/models/model_SVM_ECCnoDR_200.pkl','wb') as output_file:
#      pickle.dump(result_svm_ECC_noDR,output_file)
# with open(f'{main_folder}/models/model_SVM_AP_noDR_200.pkl','wb') as output_file:
#      pickle.dump(result_svm_AP_noDR,output_file)

with open(f'{main_folder}/models/model_QSVM_ECC_noDR_200.pkl','wb') as output_file:
     pickle.dump(result_ECC_zxzy_noDR,output_file)
with open(f'{main_folder}/models/model_QSVM_ECC_ZZZ_noDR_200.pkl','wb') as output_file:
     pickle.dump(result_ECC_zzz_noDR,output_file)

# with open(f'{main_folder}/models/model_QSVM_AP_noDR_200.pkl','wb') as output_file:
#      pickle.dump(result_AP_zxzy_noDR,output_file)
with open(f'{main_folder}/models/model_QSVM_OAR_noDR_200.pkl','wb') as output_file:
     pickle.dump(result_OAR_zxzy_noDR,output_file)


NameError: name 'main_folder' is not defined

## Running Optimal Code on IBM Q